## Urgent Data 
> take 2.0

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from doubledot import *
import pandas as pd
from time import sleep
import os

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.


In [3]:
sf = Salesforce()

In [4]:
### Clean out old data ?
# for table in Salesforce.model_d.keys():
#     sf.delete_sf_objects(table)

In [4]:
#| eval: false


# 415645 73012 105765 and 342765 are also good contacts to bring in when you are rea and organization 133156
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167, 415645, 73012, 105765, 342765 ]
pelton_ids = [ 105, 867869, 774081, 380046, 793444, 868245 ]
sf.retrieve_atms_records_by_contactId(pelton_ids)

# number of objects in dictionary
assert len(sf.atms.obj_d) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of objects in dictionary {sf.atms.obj_d.keys()}"
# number of files in directory
assert len(os.listdir(sf.atms.download_dir)) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of files in directory {sf.atms.download_dir}"



Directory 'atms_download' already exists.
my id is qsgrg8e5
fetch_data_by_contactIds : 774081
we have contact_id 774081 and obj is sales
http://crm-api-neaq.atmsplus.com/api/sales/contact/774081
[{'saleKey': '5107178', 'saleAmount': '0.0000', 'paymentAmount': '0.0000', 'saleDate': '2019-06-09T12:05:53.747', 'active': True, 'terminalKey': 23, 'ticketCount': 3, 'eventDate': '2019-06-09T21:00:00', 'booking': {'bookingId': 1152829, 'bookingContactKey': 1576060, 'bookingContactType': 'Primary', 'contactKey': 774081, 'contactIndividualKey': 719159, 'contactOrganizationKey': 0, 'displayName': 'Rodrigues, Sai', 'firstName': 'Sai', 'lastName': 'Rodrigues', 'email': 'sunnyrodrigues@gmail.com', 'phone': '8572849123'}, 'saleComment': None, 'saleDetails': [{'saleDetailId': 10661073, 'saleId': 5107178, 'itemKey': 397, 'scheduleKey': 189069, 'rateKey': 6, 'categoryKey': 33, 'itemQuantity': 3, 'pricingPriceKey': 7178, 'itemPrice': 0.0, 'itemTotal': 0.0, 'couponTotal': 0.0, 'discountTotal': 0.0, 'total

In [ ]:
#| eval: false

# perfect_data is only necessary to remove extrastuff
# perfect_data does two things, and it shouldn't: it returns a dict AND it writea a file. Pick a lane!
df_d = sf.perfect_data()
# for k, v in _d.items():
#     display(v)

In [ ]:
sf.write_dict_to_csv(df_d ) # only necessary if generate_upload_files=Fals

In [ ]:
sf.clean_upload_dir()


## Primary Test Method

In [21]:
####### PRIMARY TEST METHOD ########
sf.clean_upload_dir()
# sf.upload_csv_to_sf(clean_all=False, obj_l=['MembershipTermMemberRel__c'], generate_upload_files=True)
sf.upload_csv_to_sf(clean_all=False, generate_upload_files=True)

Salesforce.process_objects: sf_object_s : Contact
process_contacts
write_jmespath_to_csv Contact
write_jmespath_to_csv [{'LastName': 'Rodrigues', 'FirstName': 'Sai', 'MailingPostalCode': '02120', 'MailingCity': 'Boston', 'MailingStreet': '102 Hammond St Apt 2', 'MailingCountry': 'United States', 'Phone': None, 'Email': None, 'contactId__c': 774081}, {'LastName': 'Rodrigues', 'FirstName': 'Sandeep', 'MailingPostalCode': '02115-6330', 'MailingCity': 'Boston', 'MailingStreet': '779 Huntington Ave Apt 8', 'MailingCountry': 'United States', 'Phone': None, 'Email': None, 'contactId__c': 793444}, {'LastName': 'Lee', 'FirstName': 'Sherry', 'MailingPostalCode': '02467-3661', 'MailingCity': 'Chestnut Hill', 'MailingStreet': '219 Grove St', 'MailingCountry': 'United States', 'Phone': None, 'Email': None, 'contactId__c': 105}, {'LastName': 'McCain', 'FirstName': 'Andrea', 'MailingPostalCode': None, 'MailingCity': None, 'MailingStreet': None, 'MailingCountry': 'United States', 'Phone': None, 'Email

In [9]:
sf.clean_upload_dir()
sf_object_s = 'SaleDetail__c'

l_ = sf.process_saleDetails()

write_jmespath_to_csv SaleDetail__c


In [10]:
from doubledot.crema_sf import write_jmespath_to_csv
write_jmespath_to_csv(sf_object_s, l_ ,'fuck.csv', l_[0].keys())

write_jmespath_to_csv SaleDetail__c


In [51]:
l_

[{'saleDetailId__c': 10661073,
  'itemKey__c': 397,
  'scheduleKey__c': 189069,
  'rateKey__c': 6,
  'categoryKey__c': 33,
  'itemCategory__c': None,
  'pricingPriceKey__c': 7178,
  'itemPrice__c': 0.0,
  'itemTotal__c': 0.0,
  'couponTotal__c': 0.0,
  'discountTotal__c': 0.0,
  'total__c': 0.0,
  'revenueDate__c': '2019-06-09T21:00:00',
  'refundReason__c': None,
  'refundReasonKey__c': None,
  'systemPriceOverride__c': 'N',
  'membershipTermKey__r_1_membershipTermId__c': 416780,
  'saleKey__r_1_saleId__c': 5107178},
 {'saleDetailId__c': 10760392,
  'itemKey__c': 397,
  'scheduleKey__c': 189096,
  'rateKey__c': 6,
  'categoryKey__c': 33,
  'itemCategory__c': None,
  'pricingPriceKey__c': 7178,
  'itemPrice__c': 0.0,
  'itemTotal__c': 0.0,
  'couponTotal__c': 0.0,
  'discountTotal__c': 0.0,
  'total__c': 0.0,
  'revenueDate__c': '2019-07-06T21:00:00',
  'refundReason__c': None,
  'refundReasonKey__c': None,
  'systemPriceOverride__c': 'N',
  'membershipTermKey__r_1_membershipTermId__c'

[str, str, str, str, str]

In [37]:
dtype_d = { k: object for k in l_[0].keys() }
_d = pd.DataFrame(l_ )
_d.astype('int64')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [15]:
sf.clean_upload_dir()
sf.process_memberships()
sf.process_saleDetails()
sf.process_term_member_rel()

Directory 'atms_download' already exists.
Directory 'atms_upload' already exists.
write_jmespath_to_csv Membership__c
write_jmespath_to_csv [{'membershipId__c': 295573, 'memberSince__c': '2019-05-20T23:02:11.007', 'updateDate__c': '2023-01-04T15:09:22.887'}, {'membershipId__c': 295573, 'memberSince__c': '2019-05-20T23:02:11.007', 'updateDate__c': '2023-01-04T15:09:22.887'}, {'membershipId__c': 13, 'memberSince__c': '2009-09-25T00:00:00', 'updateDate__c': '2022-12-30T14:52:16.68'}, {'membershipId__c': 13, 'memberSince__c': '2009-09-25T00:00:00', 'updateDate__c': '2022-12-30T14:52:16.68'}, {'membershipId__c': 302188, 'memberSince__c': '2020-09-18T11:56:22.1', 'updateDate__c': '2022-12-09T16:04:34.39'}]
write_jmespath_to_csv for Membership__c dtypes membershipId__c    object
memberSince__c     object
updateDate__c      object
dtype: object
write_jmespath_to_csv MembershipTerm__c
write_jmespath_to_csv [{'membershipTermId__c': '416780', 'membershipKey__r_1_membershipId__c': 295573, 'effecti

In [ ]:
pd.DataFrame()

In [11]:
# sf.delete_sf_objects('MembershipTermMemberRel__c')
# assert os.path.exists(os.path.join(Salesforce.class_upload_dir,'MembershipTermMemberRel__c.csv')), 'upload file MembershipTermMemberRel__c does not exist'
sf.execute_job('MembershipTermMemberRel__c', 'insert', use_ATMS_data=True) 


execute_job
Created job 7508Y00000nJ2oMQAS for MembershipTermMemberRel__c with operation insert
Uploading job 7508Y00000nJ2oMQAS of object MembershipTermMemberRel__c
Uploaded job 7508Y00000nJ2oMQAS of object MembershipTermMemberRel__c with status 201
response.text : 
 : 
close_job response.text :
 {"id":"7508Y00000nJ2oMQAS","operation":"insert","object":"MembershipTermMemberRel__c","createdById":"0058Y00000CIEn9QAH","createdDate":"2023-05-31T03:12:06.000+0000","systemModstamp":"2023-05-31T03:12:06.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":57.0}
pre loop job status: {'id': '7508Y00000nJ2oMQAS', 'operation': 'insert', 'object': 'MembershipTermMemberRel__c', 'createdById': '0058Y00000CIEn9QAH', 'createdDate': '2023-05-31T03:12:06.000+0000', 'systemModstamp': '2023-05-31T03:12:09.000+0000', 'state': 'InProgress', 'concurrencyMode': 'Parallel', 'contentType': 'CSV', 'apiVersion': 57.0, 'jobType': 'V2Ingest', 'lineEnding': 'LF', 'columnD

## use pandas to create outer join of Members and Terms


In [ ]:

members_df = pd.read_csv('sf_upload/MembershipMember__c.csv', sep='\t')
terms_df = pd.read_csv('sf_upload/MembershipTerm__c.csv', sep='\t')
join_df = members_df.merge(terms_df, left_on='membershipTermKey__r.membershipTermId__c', right_on='membershipTermId__c', how='outer')    
# terms_df\
join_df[['membershipMemberId__c', 'membershipTermId__c']].to_csv('sf_upload/MembershipTermMemberRel__c.csv', sep='\t', index=False, index_label='Name')


In [1]:
d_l  = sf.process_saleDetails()

NameError: name 'sf' is not defined

## Problems ...


In [ ]:
#|eval: false

# why does saleId__c 889832 not have any tickets or saleDetail pointing at it? or me: 82192 323674 (I am 73012)
# why does this point from saleId and not saleKey?
from doubledot.crema_sf import fields_pointing_to_foreign_key, get_pointing_foreign_key_values,reduce_to_referenced_rows

_df = df_d['Sale__c']
print(_df.loc[_df['saleId__c'] == 323674, :])

print(get_pointing_foreign_key_values('saleId__c', df_d))
pointing = get_pointing_foreign_key_values('saleId__c', df_d)
print(len(pointing))

print(_df['saleId__c'].unique())
externals = _df['saleId__c'].unique()

pointing.intersection(externals)
      

In [ ]:
reduce_to_referenced_rows(df_d)


In [ ]:
_d['Ticket__c']['saleKey__r.saleId__c']